# Object Detection Challenge Notebook

This notebook will:
1. Set up the environment
2. Prepare the dataset
3. Train a YOLOv8 model
4. Generate predictions in the required format
5. Prepare submission files

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages
!pip install ultralytics matplotlib seaborn pandas tqdm
!pip install --upgrade ipywidgets

In [ ]:
# Import libraries
import os
import shutil
import json
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

In [ ]:
# Define paths
BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'Object_detection_RMEDU'
TRAIN_DIR = DATA_DIR / 'train'
TEST_DIR = DATA_DIR / 'test'
IMAGES_DIR = TRAIN_DIR / 'images'
LABELS_DIR = TRAIN_DIR / 'labels'
TEST_IMAGES_DIR = TEST_DIR / 'images'
SUBMISSION_DIR = BASE_DIR / 'submission'

# Create directories if they don't exist
SUBMISSION_DIR.mkdir(exist_ok=True)

# Verify paths
print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Train directory: {TRAIN_DIR}")
print(f"Test directory: {TEST_DIR}")
print(f"Submission directory: {SUBMISSION_DIR}")

# Check if directories exist
print(f"\nTrain images exist: {IMAGES_DIR.exists()}")
print(f"Train labels exist: {LABELS_DIR.exists()}")
print(f"Test images exist: {TEST_IMAGES_DIR.exists()}")

In [ ]:
# Load class names
with open('class_names.txt', 'r') as f:
    class_names = [line.strip() for line in f.readlines()]
    
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")

In [ ]:
# Explore the dataset
train_images = list(IMAGES_DIR.glob('*.jpg'))
train_labels = list(LABELS_DIR.glob('*.txt'))
test_images = list(TEST_IMAGES_DIR.glob('*.jpg'))

print(f"Number of training images: {len(train_images)}")
print(f"Number of training labels: {len(train_labels)}")
print(f"Number of test images: {len(test_images)}")

# Check if all images have corresponding labels
image_names = [img.stem for img in train_images]
label_names = [lbl.stem for lbl in train_labels]
missing_labels = set(image_names) - set(label_names)
missing_images = set(label_names) - set(image_names)

print(f"Images without labels: {len(missing_labels)}")
print(f"Labels without images: {len(missing_images)}")

In [ ]:
# Visualize some sample images with annotations
def plot_sample_images(num_samples=3):
    plt.figure(figsize=(15, 10))
    
    for i in range(num_samples):
        # Select a random image
        img_idx = random.randint(0, len(train_images)-1)
        img_path = train_images[img_idx]
        label_path = LABELS_DIR / f"{img_path.stem}.txt"
        
        # Read image
        img = plt.imread(img_path)
        h, w, _ = img.shape
        
        # Read annotations
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        # Plot image
        plt.subplot(1, num_samples, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Sample {i+1}")
        
        # Plot bounding boxes
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            
            # Convert normalized coordinates to pixel values
            x_center *= w
            y_center *= h
            width *= w
            height *= h
            
            # Calculate corner points
            x1 = int(x_center - width/2)
            y1 = int(y_center - height/2)
            x2 = int(x_center + width/2)
            y2 = int(y_center + height/2)
            
            # Draw rectangle
            plt.gca().add_patch(plt.Rectangle(
                (x1, y1), width, height,
                fill=False, color='red', linewidth=2
            ))
            
            # Add class name
            plt.text(
                x1, y1-5, class_names[int(class_id)],
                color='red', fontsize=10, backgroundcolor='white'
            )
    
    plt.tight_layout()
    plt.show()

plot_sample_images()

In [ ]:
# Create data.yaml file for YOLOv8
data_yaml = f"""
path: {DATA_DIR.absolute()}
train: train/images
val: train/images  # Using training data as validation since no separate validation set
test: test/images

nc: {num_classes}
names: {class_names}
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)

print("Created data.yaml file:")
print(data_yaml)

In [ ]:
# Initialize YOLOv8 model
model = YOLO('yolov8n.pt')  # Using pretrained nano model for faster training

# Train the model
results = model.train(
    data='data.yaml',
    epochs=50,  # Adjust based on time constraints
    imgsz=640,
    batch=16,   # Adjust based on GPU memory
    name='object_detection_model',
    project='runs/train',
    exist_ok=True,
    verbose=True,
    device=0,   # Use GPU
    patience=5, # Early stopping
    save_period=5  # Save every 5 epochs
)

In [ ]:
# Plot training results
def plot_training_results():
    # Find the latest training run
    train_dir = Path('runs/train')
    latest_run = max([d for d in train_dir.glob('object_detection_model*') if d.is_dir()], key=os.path.getmtime)
    results_file = latest_run / 'results.csv'
    
    if not results_file.exists():
        print("No results.csv found. Training might not have completed.")
        return
    
    # Read results
    results = pd.read_csv(results_file)
    
    # Plot metrics
    plt.figure(figsize=(15, 10))
    
    # Plot training and validation loss
    plt.subplot(2, 2, 1)
    plt.plot(results['epoch'], results['train/box_loss'], label='train/box_loss')
    plt.plot(results['epoch'], results['val/box_loss'], label='val/box_loss')
    plt.title('Box Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(2, 2, 2)
    plt.plot(results['epoch'], results['train/cls_loss'], label='train/cls_loss')
    plt.plot(results['epoch'], results['val/cls_loss'], label='val/cls_loss')
    plt.title('Class Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot mAP metrics
    plt.subplot(2, 2, 3)
    plt.plot(results['epoch'], results['metrics/mAP50(B)'], label='mAP@50')
    plt.plot(results['epoch'], results['metrics/mAP50-95(B)'], label='mAP@50-95')
    plt.title('mAP Metrics')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.legend()
    
    # Plot precision and recall
    plt.subplot(2, 2, 4)
    plt.plot(results['epoch'], results['metrics/precision(B)'], label='precision')
    plt.plot(results['epoch'], results['metrics/recall(B)'], label='recall')
    plt.title('Precision and Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_results()

In [ ]:
# Load the best model
best_model_path = 'runs/train/object_detection_model/weights/best.pt'
model = YOLO(best_model_path)

# Make predictions on test set
print("Generating predictions on test set...")
test_images_paths = [str(p) for p in TEST_IMAGES_DIR.glob('*.jpg')]

# Clear submission directory
for f in SUBMISSION_DIR.glob('*.txt'):
    f.unlink()

# Predict on each test image
for img_path in tqdm(test_images_paths):
    # Get image filename without extension
    img_name = Path(img_path).stem
    
    # Make prediction
    results = model.predict(img_path, conf=0.25, iou=0.45, verbose=False)
    
    # Create output file
    output_file = SUBMISSION_DIR / f"{img_name}.txt"
    
    # Write predictions to file
    with open(output_file, 'w') as f:
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Get class ID and normalized coordinates
                    cls = int(box.cls)
                    x_center, y_center, width, height = box.xywhn[0].tolist()
                    
                    # Write in required format: <class_id> <x_center> <y_center> <width> <height>
                    f.write(f"{cls} {x_center} {y_center} {width} {height}\n")

print(f"Generated {len(list(SUBMISSION_DIR.glob('*.txt')))} prediction files")

In [ ]:
# Verify submission format
sample_files = list(SUBMISSION_DIR.glob('*.txt'))[:5]
for file in sample_files:
    print(f"File: {file.name}")
    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines[:3]:  # Show first 3 lines
            print(line.strip())
        if len(lines) > 3:
            print(f"... and {len(lines)-3} more lines")
        print()

In [ ]:
# Create submission archive
submission_archive = 'submission.zip'
shutil.make_archive(submission_archive.replace('.zip', ''), 'zip', SUBMISSION_DIR)

print(f"Submission archive created: {submission_archive}")
print(f"Archive size: {os.path.getsize(submission_archive) / (1024*1024):.2f} MB")

# Verify archive contents
with zipfile.ZipFile(submission_archive, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    print(f"Number of files in archive: {len(file_list)}")
    print("Sample files:")
    for f in file_list[:5]:
        print(f"  - {f}")

In [ ]:
# Optional: Visualize predictions on test images
def visualize_predictions(num_samples=3):
    plt.figure(figsize=(15, 10))
    
    # Get random test images
    sample_files = random.sample(test_images_paths, num_samples)
    
    for i, img_path in enumerate(sample_files):
        img_name = Path(img_path).stem
        
        # Load image
        img = plt.imread(img_path)
        h, w, _ = img.shape
        
        # Load predictions
        pred_file = SUBMISSION_DIR / f"{img_name}.txt"
        
        # Plot image
        plt.subplot(1, num_samples, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Prediction: {img_name}")
        
        if pred_file.exists():
            with open(pred_file, 'r') as f:
                lines = f.readlines()
            
            for line in lines:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                
                # Convert normalized coordinates to pixel values
                x_center *= w
                y_center *= h
                width *= w
                height *= h
                
                # Calculate corner points
                x1 = int(x_center - width/2)
                y1 = int(y_center - height/2)
                x2 = int(x_center + width/2)
                y2 = int(y_center + height/2)
                
                # Draw rectangle
                plt.gca().add_patch(plt.Rectangle(
                    (x1, y1), width, height,
                    fill=False, color='lime', linewidth=2
                ))
                
                # Add class name
                plt.text(
                    x1, y1-5, class_names[int(class_id)],
                    color='lime', fontsize=10, backgroundcolor='black'
                )
    
    plt.tight_layout()
    plt.show()

visualize_predictions()

## Summary

This notebook has:
1. Set up the environment and installed necessary packages
2. Explored and visualized the dataset
3. Created a data.yaml configuration file for YOLOv8
4. Trained a YOLOv8 model on the training data
5. Generated predictions on the test set in the required format
6. Created a submission archive with all prediction files

The submission files are in the `submission` directory and packaged as `submission.zip`.

To submit to the competition:
1. Download the `submission.zip` file
2. Extract it to get all the prediction files
3. Upload the individual .txt files to the competition platform

Each .txt file corresponds to a test image and contains the predicted bounding boxes in the format:
`<class_id> <x_center> <y_center> <width> <height>`